# Loading Dependencies

## Loading Data

In [1]:
from sklearn.datasets import fetch_20newsgroups
newsgroup = fetch_20newsgroups(subset='all')

## Importing Libraries

In [2]:
import nltk
import string
from tqdm.auto import tqdm
from nltk.tokenize import word_tokenize

## Preprocessing Dependencies

In [3]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = nltk.stem.SnowballStemmer('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

In [4]:
def preprocess_article(article):
    article = word_tokenize(article.lower().strip())
    article = [
        lemmatizer.lemmatize(w)
        for w in article 
            if w not in stopwords and w not in string.punctuation
    ]
    return article

## BST Dependencies

In [5]:
class Node:
    def __init__(self, word, document_id):
        self.word = word
        self.postings = [document_id]
        self.document_frequency = 1

        self.left = None
        self.right = None

    def add_doc_to_word_posting(self, document_id):
        if document_id not in self.postings:
            self.postings.append(document_id)
            self.document_frequency += 1

In [6]:
class BST:
    def __init__(self):
        self.root = None

    def insert(self, word, document_id):
        if self.root is None:
            self.root = Node(word, document_id)
            return
        else:
            self.insert_word(self.root, word, document_id)
    
    def insert_word(self, node, word, document_id):
        if word < node.word:
            if node.left is None:
                node.left = Node(word, document_id)
            else:
                self.insert_word(node.left, word, document_id)
        elif word > node.word:
            if node.right is None:
                node.right = Node(word, document_id)
            else:
                self.insert_word(node.right, word, document_id)
        else:
            node.add_doc_to_word_posting(document_id)

    def search(self, word):
        if self.root:
            return self.search_word(self.root, word)
        else:
            return None

    def search_word(self, node, word):
        if word < node.word:
            if node.left is None:
                return None
            else:
                return self.search_word(node.left, word)
        elif word > node.word:
            if node.right is None:
                return None
            else:
                return self.search_word(node.right, word)
        else:
            return (node.postings, node.document_frequency)

    def inorder(self):
        if self.root:
            self.inorder_traversal(self.root)

    def inorder_traversal(self, node):
        if node.left:
            self.inorder_traversal(node.left)
        print(f'{node.word} {node.postings} {node.document_frequency}')
        if node.right:
            self.inorder_traversal(node.right)

# Creating BST

In [7]:
def create_search_tree():
    search_tree = BST()
    for i in tqdm(range(len(newsgroup['data']))):
        article = newsgroup['data'][i]
        article = preprocess_article(article)
        document_id = newsgroup['target'][i]
        for word in article:
            search_tree.insert(word, document_id)
    return search_tree

In [8]:
%%time
search_tree = create_search_tree()

  0%|          | 0/18846 [00:00<?, ?it/s]

CPU times: user 1min 23s, sys: 433 ms, total: 1min 23s
Wall time: 1min 22s


# Querying BST

In [9]:
# def query_search_tree(query): # To handle multi-word queries
#     query = preprocess_article(query)
#     query_result = list()
#     for word in query:
#         word_result = search_tree.search(word)
#         query_result.append(set(word_result[0]))
#     return list(set.intersection(*query_result))

In [10]:
%%time
search_result = search_tree.search("india")
search_result

CPU times: user 58 µs, sys: 2 µs, total: 60 µs
Wall time: 64.4 µs


([17, 19, 1, 13, 15, 14, 18, 0, 11], 9)